In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
import folium
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext , functions
from folium import plugins


In [ ]:
spark = SparkSession.builder \
.appName('ReadFromParquet') \
.master('local[6]') \
.getOrCreate()

In [ ]:
spark

In [ ]:
sc = spark.sparkContext
sqlContext = SQLContext(sc)

# Data Feature

## osmWay

In [ ]:
sqlContext.setConf("spark.sql.parquet.binaryAsString","true")
osmWay = sqlContext.read.parquet("../data/20190531-hungary.osm.pbf.way.parquet")
osmWay = osmWay.select('id','tags','nodes')
osmWay.createOrReplaceTempView("osmWay")




In [ ]:
osmWay.printSchema()

In [ ]:
osmWay.head()

In [ ]:
osmWay.show(5)

In [ ]:
osmWay.count()

## osmNode

In [ ]:
osmNode = sqlContext.read.parquet("../data/20190531-hungary.osm.pbf.node.parquet")
osmNode = osmNode.select('tags', 'latitude','longitude')
osmNode.createOrReplaceTempView("osmNode")

In [ ]:
osmNode.printSchema()

In [ ]:
osmNode.head()

In [ ]:
osmNode.show(5)

In [ ]:
osmNode.count()

In [ ]:
sqlContext.sql("SELECT tags,latitude, longitude FROM osmNode \
WHERE tags[0].key='amenity' AND tags[0].value='taxi'").show()

## osmRelation

In [ ]:
osmRelation = sqlContext.read.parquet("../data/20190531-hungary.osm.pbf.relation.parquet")
osmRelation = osmRelation.select('id','tags','members')
osmRelation.createOrReplaceTempView("osmRelation")

In [ ]:
osmRelation.printSchema()

In [ ]:
osmRelation.head()

In [ ]:
osmRelation.show(5)

In [ ]:
osmRelation.count()

# Data Explore

In [ ]:
## Number of Taxi Stop and Location Each of Them

In [ ]:
taxiLocation = sqlContext.sql("SELECT latitude,longitude FROM osmNode \
WHERE tags[0].key='amenity' AND tags[0].value='taxi'").show()

In [ ]:
taxiLocation = sqlContext.sql("SELECT latitude,longitude FROM osmNode \
WHERE tags[0].key='amenity' AND tags[0].value='taxi'").toPandas()

In [ ]:
taxidf = pd.DataFrame(data=taxiLocation)
taxidf

In [ ]:
taxiMap = folium.Map([taxiLocation.latitude[0], taxiLocation.longitude[0]], zoom_start=7)

In [ ]:
for index, row in taxidf.iterrows():
    folium.CircleMarker([row['latitude'], row['longitude']],
                        radius=15,
                        popup=row[['latitude','longitude']],
                        fill_color="#e4d93d", # divvy color
                       ).add_to(taxiMap)
taxiMap